## 다중 분류 문제
- 난방 부하 단계를 예측해주세요!

- 예측할 값(y): Heat_Load (Very Low, Low, Medium, High, Very High)
- 평가: f1-macro
- data: train.csv, test.csv
- 제출 형식: result.csv파일을 아래와 같은 형식으로 제출
~~~
pred
Very Low
Low
High
...
Very High
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

In [33]:
import pandas as pd
train = pd.read_csv('energy_train.csv')
test = pd.read_csv('energy_test.csv')

train.shape, test.shape

((537, 10), (231, 9))

In [34]:
train.head() #Heat_Load를 예측. test에서 예측 필요

,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load,Heat_Load
0,0.74,686.0,245.0,220.5,Short,South,0.25,3,14.72,Very Low
1,0.98,514.5,294.0,Small,Tall,South,0.40,2,33.94,High
2,0.90,563.5,318.5,Medium,Tall,West,0.25,1,37.58,High
3,0.74,686.0,245.0,220.5,Short,South,0.25,2,15.18,Very Low
4,0.98,514.5,294.0,Small,Tall,East,0.25,2,29.69,High


In [35]:
test.head()

,Compac,Surf_Area,Wall_Area,Roof,Height,Orient,Glaze_Area,Glaze_Distr,Cool_Load
0,0.64,784.0,343.0,220.5,Short,South,0.40,4,22.25
1,0.82,612.5,318.5,Large,Tall,North,0.40,3,32.43
2,0.76,661.5,416.5,Medium,Tall,South,0.10,5,33.64
3,0.74,686.0,245.0,220.5,Short,East,0.40,1,17.25
4,0.64,784.0,343.0,220.5,Short,North,0.25,5,20.13


In [36]:
# 결측값이 있는지 확인
print(train.isnull().sum())
print(test.isnull().sum())
print(train['Heat_Load'].value_counts())

Compac         0
Surf_Area      0
Wall_Area      0
Roof           0
Height         0
Orient         0
Glaze_Area     0
Glaze_Distr    0
Cool_Load      0
Heat_Load      0
dtype: int64
Compac         0
Surf_Area      0
Wall_Area      0
Roof           0
Height         0
Orient         0
Glaze_Area     0
Glaze_Distr    0
Cool_Load      0
dtype: int64
Heat_Load
Very Low     142
Low          123
High         122
Very High     79
Medium        71
Name: count, dtype: int64


In [41]:
# 범주형과 수치형을 확인
train.info() # 수치형 6개 / 범주형 4개
n_train = train.select_dtypes(exclude = 'object')
# train_heatload = train.pop('Heat_Load')
train = pd.get_dummies(train) #나머지 오브젝트 값들은 원핫인코딩을 진행함.
test = pd.get_dummies(test)
print(train_heatload) # 검증용이므로 따로 뺄것.
print(train.shape, test.shape)
c_train = train.select_dtypes(include = 'object')

#print(n_train.info(), c_train.info())
#print(train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Compac        537 non-null    float64
 1   Surf_Area     537 non-null    float64
 2   Wall_Area     537 non-null    float64
 3   Glaze_Area    537 non-null    float64
 4   Glaze_Distr   537 non-null    int64  
 5   Cool_Load     537 non-null    float64
 6   Roof_220.5    537 non-null    bool   
 7   Roof_Large    537 non-null    bool   
 8   Roof_Medium   537 non-null    bool   
 9   Roof_Small    537 non-null    bool   
 10  Height_Short  537 non-null    bool   
 11  Height_Tall   537 non-null    bool   
 12  Orient_East   537 non-null    bool   
 13  Orient_North  537 non-null    bool   
 14  Orient_South  537 non-null    bool   
 15  Orient_West   537 non-null    bool   
dtypes: bool(10), float64(5), int64(1)
memory usage: 30.5 KB
0       Very Low
1           High
2           High
3       V

In [43]:
# 테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, train_heatload, test_size = 0.2, random_state = 2024)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((429, 16), (108, 16), (429,), (108,))

In [46]:
# 분리 후 테스트 데이터 검증 실시
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(random_state = 2024)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val) # 확률이 아닌 다섯개의 범주형 값이므로 predict 사용
print(pred)

print(f1_score(y_val, pred, average = 'macro')) #y_val / test

['Low' 'Low' 'Very High' 'Very Low' 'Low' 'Very High' 'Medium' 'High'
 'Low' 'Very Low' 'Very High' 'High' 'Very Low' 'High' 'Very Low'
 'Very Low' 'Very Low' 'Medium' 'Very Low' 'High' 'Very High' 'Very Low'
 'Very Low' 'Very Low' 'Low' 'High' 'Low' 'High' 'Very High' 'High'
 'Very Low' 'Very High' 'Very Low' 'Very Low' 'Medium' 'High' 'High'
 'Very Low' 'High' 'Very Low' 'Low' 'High' 'Medium' 'Low' 'Very Low'
 'High' 'Very High' 'Very High' 'Low' 'Very Low' 'Very Low' 'Very Low'
 'Very High' 'High' 'Low' 'Medium' 'Low' 'Very Low' 'Very High' 'Low'
 'High' 'Very Low' 'High' 'High' 'High' 'Very Low' 'Very High' 'Very High'
 'Low' 'Low' 'Low' 'High' 'Very Low' 'Low' 'High' 'High' 'Low' 'Low'
 'Very Low' 'Very Low' 'Medium' 'Very High' 'Very High' 'Low' 'Low'
 'Very High' 'Low' 'High' 'Very Low' 'High' 'Very Low' 'Low' 'Very High'
 'Very Low' 'Low' 'Low' 'Very High' 'Medium' 'Very Low' 'Low' 'Very Low'
 'Very Low' 'High' 'Very Low' 'Medium' 'Very Low' 'Low' 'Low']
0.9532030572855499


In [48]:
# test 데이터를 통한 예측과 csv 변환
pred = rf.predict(test)
submit = pd.DataFrame({
    'pred': pred
})
print(submit)

submit.to_csv('result.csv', index = False)

          pred
0          Low
1         High
2         High
3          Low
4          Low
..         ...
226   Very Low
227     Medium
228   Very Low
229        Low
230  Very High

[231 rows x 1 columns]
